# Worked in GROUP - 16


Name: **SatyaTej Kammili**


Student ID : **11911161**


Contribution Made: R2, R5 and Github



Name: **Merla Ganesh Reddy**


Student ID: **11797294**


Contribution Made: R3 and R4

# Importing required libraries

In [1]:
!pip install --quiet networkx tqdm

import os, sys, random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

print("Python:", sys.version.split()[0])
print("NetworkX:", nx.__version__)

Python: 3.12.11
NetworkX: 3.5


# Uploading "Project 1 Part B Fall 2025.docx

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print("Uploaded file:", fn)


Saving Project 1 Part B Fall 2025.docx to Project 1 Part B Fall 2025.docx
Uploaded file: Project 1 Part B Fall 2025.docx


# Installing python-docx & Reading the document

In [3]:

!pip install --quiet python-docx

from docx import Document


doc = Document("Project 1 Part B Fall 2025.docx")

print("---- Document Content ----\n")
for para in doc.paragraphs:
    if para.text.strip():
        print(para.text)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 8.2 MB/s eta 0:00:00
---- Document Content ----

Project 1 Artificial Intelligence Fall 2025
Distributed: Wednesday 27 August 2025
Due Part A: Friday 5 September 2025
Part B: Sunday 21 September 2025
This project may be undertaken either individually or in pairs. Please state the names of the (person or persons undertaking the project and the contributions that each has made. Only ONE submission must be made per group.
Purpose:  
To gain a thorough understanding of the working of an agent to assist in navigation in a stochastic setting that mimics a real-world environment. Thus, this requires reading through the project specification carefully to make sure that you understand its requirements. Jot down any questions/doubts that you may have and feel free to ask me questions in class or in person. Together with your partner, work out a strategy before you start coding the solution in Python. Given the limited timeframe for the p

# Creating fixed network graph & Adding Nodes for R2

In [4]:
G = nx.Graph()


nodes = [0, 1, 2, 3, 4]
G.add_nodes_from(nodes)


edges = [
    (0, 2, 10),
    (0, 4, 15),
    (1, 2, 12),
    (3, 4, 8)
]
G.add_weighted_edges_from(edges)


print("Nodes:", G.nodes())
print("Edges with weights:", G.edges(data=True))


Nodes: [0, 1, 2, 3, 4]
Edges with weights: [(0, 2, {'weight': 10}), (0, 4, {'weight': 15}), (1, 2, {'weight': 12}), (3, 4, {'weight': 8})]


# Simulating trips and computing shortest paths

In [5]:
T = 100
ticks = 1

for u, v in G.edges():
    G[u][v]['trips'] = 0


def get_path_edges(path):
    return [(path[i], path[i+1]) for i in range(len(path)-1)]


for _ in range(ticks):
    for _ in range(T):
        start, end = random.sample(nodes, 2)
        path = nx.astar_path(G, start, end, weight='weight')
        for u, v in get_path_edges(path):
            G[u][v]['trips'] += 1


for u, v, data in G.edges(data=True):
    print(f"Edge ({u},{v}) - Weight: {data['weight']}, Trips: {data['trips']}")


Edge (0,2) - Weight: 10, Trips: 62
Edge (0,4) - Weight: 15, Trips: 58
Edge (1,2) - Weight: 12, Trips: 37
Edge (3,4) - Weight: 8, Trips: 41


# Computing the benefit of proposed new roads

Formula using

Benefit =
(
Current shortest path length
−
New road length
)
×
Sum of trips along the path

Using shrinkage factor f = 0.6 to estimate the new road lengths

In [6]:
candidates = [(0,1), (0,3), (1,3), (1,4), (2,4)]
f = 0.6
benefits = {}
for u, v in candidates:
    sp_length = nx.astar_path_length(G, u, v, weight='weight')
    new_road_length = f * sp_length

    path = nx.astar_path(G, u, v, weight='weight')
    trips_sum = sum(G[path[i]][path[i+1]]['trips'] for i in range(len(path)-1))
    benefit = (sp_length - new_road_length) * trips_sum
    benefits[(u,v)] = benefit

for road, value in benefits.items():
    print(f"Road {road} - Benefit: {value:.2f}")


Road (0, 1) - Benefit: 871.20
Road (0, 3) - Benefit: 910.80
Road (1, 3) - Benefit: 3564.00
Road (1, 4) - Benefit: 2323.60
Road (2, 4) - Benefit: 1200.00


# Which two roads to build

In [7]:
G_build = G.copy()
remaining_candidates = candidates.copy()

# First road
first_road = max(benefits, key=benefits.get)
print(f"First road to build: {first_road} - Benefit: {benefits[first_road]:.2f}")


sp_length_first = nx.astar_path_length(G_build, first_road[0], first_road[1], weight='weight')
new_length_first = f * sp_length_first
G_build.add_edge(first_road[0], first_road[1], weight=new_length_first, trips=0)


remaining_candidates.remove(first_road)

# Recompute benefits
new_benefits = {}
for u, v in remaining_candidates:
    sp_length = nx.astar_path_length(G_build, u, v, weight='weight')
    new_road_length = f * sp_length
    path = nx.astar_path(G_build, u, v, weight='weight')
    trips_sum = sum(G_build[path[i]][path[i+1]]['trips'] for i in range(len(path)-1))
    benefit = (sp_length - new_road_length) * trips_sum
    new_benefits[(u,v)] = benefit

# Second road
second_road = max(new_benefits, key=new_benefits.get)
print(f"Second road to build: {second_road} - Benefit: {new_benefits[second_road]:.2f}")


First road to build: (1, 3) - Benefit: 3564.00
Second road to build: (2, 4) - Benefit: 1200.00


# Results:

For R2:

First road to build: (1, 3) – Benefit: 3564.00

Second road to build: (2, 4) – Benefit: 1200.00

# R3 – Random 60-node network

*   Creating a random graph with 60 nodes.
*   Using a connectivity p = 0.05.
*   Running the same traffic simulation and computing the top 3 roads to build






In [8]:
import numpy as np

N = 60
p = 0.05


connected = False
while not connected:
    G_random = nx.gnp_random_graph(N, p)
    connected = nx.is_connected(G_random)
    if not connected:
        p += 0.01


for u, v in G_random.edges():
    G_random[u][v]['weight'] = random.randint(5, 25)
    G_random[u][v]['trips'] = 0

print(f"Random graph created with p={p:.2f} and {G_random.number_of_edges()} edges.")


Random graph created with p=0.06 and 113 edges.


# Simulating trips on the random graph

*   T = 100 trips per tick
*   ticks = 1
*   Updating trips on each edge









In [9]:
T = 100
ticks = 1
nodes_random = list(G_random.nodes())

for u, v in G_random.edges():
    G_random[u][v]['trips'] = 0

def get_path_edges(path):
    return [(path[i], path[i+1]) for i in range(len(path)-1)]

for _ in range(ticks):
    for _ in range(T):
        start, end = random.sample(nodes_random, 2)
        path = nx.astar_path(G_random, start, end, weight='weight')
        for u, v in get_path_edges(path):
            G_random[u][v]['trips'] += 1

sample_edges = list(G_random.edges(data=True))[:10]
for u, v, data in sample_edges:
    print(f"Edge ({u},{v}) - Weight: {data['weight']}, Trips: {data['trips']}")


Edge (0,3) - Weight: 5, Trips: 7
Edge (0,11) - Weight: 22, Trips: 2
Edge (0,15) - Weight: 17, Trips: 1
Edge (0,32) - Weight: 21, Trips: 1
Edge (1,6) - Weight: 25, Trips: 1
Edge (1,20) - Weight: 19, Trips: 1
Edge (1,39) - Weight: 17, Trips: 3
Edge (2,33) - Weight: 21, Trips: 2
Edge (2,46) - Weight: 11, Trips: 2
Edge (3,15) - Weight: 17, Trips: 0


Computing benefits and recommend top 3 roads (R3)



*   Generate candidate new roads: any pair of nodes not currently connected.
*   Computing the benefits for each candidate using f = 0.6.
*   Select the top 3 roads to build










In [10]:
f = 0.6
k = 3


candidates = [(u,v) for u in nodes_random for v in nodes_random if u < v and not G_random.has_edge(u,v)]


def compute_benefit(G, u, v, f):
    sp_length = nx.astar_path_length(G, u, v, weight='weight')
    new_length = f * sp_length
    path = nx.astar_path(G, u, v, weight='weight')
    trips_sum = sum(G[path[i]][path[i+1]]['trips'] for i in range(len(path)-1))
    return (sp_length - new_length) * trips_sum


G_build = G_random.copy()
recommended = []

for _ in range(k):
    benefits = {edge: compute_benefit(G_build, edge[0], edge[1], f) for edge in candidates}
    best_road = max(benefits, key=benefits.get)
    recommended.append((best_road, benefits[best_road]))

    sp_length = nx.astar_path_length(G_build, best_road[0], best_road[1], weight='weight')
    G_build.add_edge(best_road[0], best_road[1], weight=f*sp_length, trips=0)

    candidates.remove(best_road)


for i, (road, benefit) in enumerate(recommended, 1):
    print(f"Top {i} road to build: {road} - Benefit: {benefit:.2f}")


Top 1 road to build: (8, 9) - Benefit: 2530.00
Top 2 road to build: (7, 35) - Benefit: 2038.40
Top 3 road to build: (35, 53) - Benefit: 2035.20


Results:

*For R3*


Top 3 roads to build (f = 0.6):



1.   (8, 9) – Benefit: 2530.00
2.   (7, 35) – Benefit: 2038.40
3.   (35, 53) – Benefit: 2035.20







**R4 – Updating shrinkage factor f = 0.8 and recomputing benefits**

In [11]:
f = 0.8
k = 3
G_build = G_random.copy()
candidates = [(u,v) for u in nodes_random for v in nodes_random if u < v and not G_build.has_edge(u,v)]
recommended_f08 = []

for _ in range(k):
    benefits = {edge: compute_benefit(G_build, edge[0], edge[1], f) for edge in candidates}
    best_road = max(benefits, key=benefits.get)
    recommended_f08.append((best_road, benefits[best_road]))

    sp_length = nx.astar_path_length(G_build, best_road[0], best_road[1], weight='weight')
    G_build.add_edge(best_road[0], best_road[1], weight=f*sp_length, trips=0)

    candidates.remove(best_road)


for i, (road, benefit) in enumerate(recommended_f08, 1):
    print(f"Top {i} road to build (f=0.8): {road} - Benefit: {benefit:.2f}")


Top 1 road to build (f=0.8): (8, 9) - Benefit: 1265.00
Top 2 road to build (f=0.8): (9, 11) - Benefit: 1067.00
Top 3 road to build (f=0.8): (8, 57) - Benefit: 1029.60


# Results for R4:

Top 3 roads to build (f = 0.8):



1.   (8, 9) – Benefit: 1265.00
2.   (9, 11) – Benefit: 1067.00
3.   (8, 57) – Benefit: 1029.60

**Observation**: The recommendations changed from R3. This happens because increasing f increases the length of the new road relative to the old shortest path (new_length = f * sp_length), which reduces the benefit for some roads and changes the ranking.

# R5 – Double the connectivity and recomputing top 3 roads

For R5:


1.   Double p from R3 (p = 0.12).
2.   Generate a new random connected graph with 60 nodes.
3.   Simulate traffic, compute benefits, and get the top 3 roads.



In [12]:
p_new = 0.12
connected = False
while not connected:
    G_new = nx.gnp_random_graph(N, p_new)
    connected = nx.is_connected(G_new)
    if not connected:
        p_new += 0.01


for u, v in G_new.edges():
    G_new[u][v]['weight'] = random.randint(5, 25)
    G_new[u][v]['trips'] = 0


nodes_new = list(G_new.nodes())
for _ in range(ticks):
    for _ in range(T):
        start, end = random.sample(nodes_new, 2)
        path = nx.astar_path(G_new, start, end, weight='weight')
        for i in range(len(path)-1):
            u, v = path[i], path[i+1]
            G_new[u][v]['trips'] += 1

f = 0.6
candidates_new = [(u,v) for u in nodes_new for v in nodes_new if u < v and not G_new.has_edge(u,v)]
G_build = G_new.copy()
recommended_new = []

for _ in range(k):
    benefits = {edge: compute_benefit(G_build, edge[0], edge[1], f) for edge in candidates_new}
    best_road = max(benefits, key=benefits.get)
    recommended_new.append((best_road, benefits[best_road]))
    sp_length = nx.astar_path_length(G_build, best_road[0], best_road[1], weight='weight')
    G_build.add_edge(best_road[0], best_road[1], weight=f*sp_length, trips=0)
    candidates_new.remove(best_road)


for i, (road, benefit) in enumerate(recommended_new, 1):
    print(f"Top {i} road to build with doubled connectivity: {road} - Benefit: {benefit:.2f}")


Top 1 road to build with doubled connectivity: (11, 19) - Benefit: 644.00
Top 2 road to build with doubled connectivity: (9, 18) - Benefit: 630.00
Top 3 road to build with doubled connectivity: (5, 24) - Benefit: 570.00


Results for R5.

*   Top 3 roads with doubled connectivity (p ≈ 0.12):




1.   (11, 19) – Benefit: 644.00
2.   (9, 18) – Benefit: 630.00
3.   (5, 24) – Benefit: 570.00



*Observation*: The benefits decreased significantly compared to the lower connectivity network in R3. This happens because:


*   Higher connectivity means there are more existing roads, so the shortest paths are shorter.

*   Consequently, adding a new road reduces the path length less dramatically, lowering the benefit.

*   The top candidate roads are different because the network topology changed, creating new bottlenecks in different places.








